### Set up LLM and Embedding

In [3]:
#Install prerequisite packages
!pip install llama-index -q
!pip install llama-index-llms-groq -q
!pip install llama-index-embeddings-huggingface -q

In [4]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

from google.colab import userdata

#Setup the LLM
Settings.llm=Groq(model="llama-3.3-70b-versatile", api_key=userdata.get('GROQ_API_KEY'))

#Setup the embedding model RAG
Settings.embed_model= HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 02.03. Set up wikipedia tools

In [7]:
!pip install llama-index-tools-wikipedia -q

  Preparing metadata (setup.py) ... done


In [8]:
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.tools.tool_spec.load_and_search import (
    LoadAndSearchToolSpec,
)

# Get list of wikipedia tools
wiki_spec = WikipediaToolSpec()
tool = wiki_spec.to_tool_list()[1]
wiki_tools = LoadAndSearchToolSpec.from_defaults(tool).to_tool_list()

#Review the list of tools
for tool in wiki_tools:
    print( "--------\n",tool.metadata.name, tool.metadata.description)

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    


### 02.04. Set up RAG for medications

In [10]:
!pip install llama-index-readers-json -q

In [11]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.readers.json import JSONReader

medication_docs = JSONReader().load_data(
        input_file="Medications and dosages.json"
)
#Split the document
splitter=SentenceSplitter(chunk_size=200)
medication_nodes=splitter.get_nodes_from_documents(medication_docs)

#Index the document in memory
medication_index=VectorStoreIndex(medication_nodes)

#Create a query engine tool based on the CSV file
medication_query_engine = medication_index.as_query_engine()
medication_tool = QueryEngineTool.from_defaults(
    query_engine=medication_query_engine,
    description=(
        "Provides list of popular medications, diseases they are used to treat for and recommended dosages"
    ),
)

### 02.05. Set up & execute the Healthcare Agent

In [12]:
from llama_index.core.agent.react.base import ReActAgent
#Consolidate list of tools
healthcare_tools =wiki_tools + [medication_tool]

context = """
You are an healthcare assistant who can answer questions regarding
diseases, their symptoms and medications for them.

Use the wikipedia tools to answer questions about symptoms and possible
diseases associated with those symptoms.
Use the medication tool to find list of medications and dosages for a given
disease.

Use only the tools provided to answer questions and NOT your own memory.

"""

#Create a pre-built ReAct agent
healthcare_agent = ReActAgent.from_tools(
                healthcare_tools,
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


In [13]:
response=healthcare_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 58a9c681-b830-44a2-b992-1957e2797490. Step input: Which medication should i take for arthritis?
Thought: The user is asking about medication for arthritis. To answer this, I need to use the query_engine_tool to find a list of medications and their dosages for arthritis.
Action: query_engine_tool
Action Input: {'input': 'arthritis'}
Observation: None of the listed medications are used for arthritis. The medications listed are used for hypertension, fluid retention, benign prostatic hyperplasia, epilepsy, migraines, moderate to severe pain, depression, and anxiety.
> Running step 8ff04be4-c8e5-41fa-bd47-3341d5cd83ec. Step input: None
Thought: The query_engine_tool did not provide the expected results for arthritis medication. I need to try again with a more specific query to get the correct list of medications for arthritis.
Action: query_engine_tool
Action Input: {'input': 'medications for arthritis'}
Observation: There is no information about medications for arthritis. T